In [ ]:
import h5py
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def read_step(step: int, electronsOnly: bool = True):
    rank = 0
    df = pd.read_hdf(path + f"prt.{step:06d}_p{rank:06d}.h5", 'particles/p0/1d')
    df.drop(columns=["x", "px", "m", "w", "tag"], inplace=True)
    if electronsOnly:
        df = df[df.q == -1]
        df.drop(columns=["q"], inplace=True)
    df["step"] = step
    return df

def plot_distribution(df, x: str, y: str):
    fig, ax = plt.subplots(1, 1)
    df.plot.hexbin(x, y, gridsize=50, ax=ax);
    ax.set_title(f"f({x}, {y}) at t={t} for B={B}")
    if x.endswith(tuple("y, z")) and y.endswith(tuple("y, z")):
        ax.set_aspect(1.)

## View paths of sample particles

In [ ]:
path = '/mnt/lustre/IAM851/jm1667/psc-runs/case1/single/B.1_n512_v+/'

steps = range(0, 100000, 10000)
dfs = [read_step(steps[0])[::100000]]
ids = dfs[0].id
for step in steps[1:]:
    df = read_step(step)
    dfs.append(df[df.id.isin(ids)])

dfs = pd.concat(dfs)
dfs.sort_values(by=["id"]);

In [ ]:
%matplotlib widget
dfs.plot.scatter(x="y", y="z", marker='.', c="id")
# plt.quiver(dfs.y, dfs.z, dfs.py, dfs.pz)
plt.gca().set_aspect(1)

In [ ]:
step = 10000
df = read_step(step).iloc[::100]
df.plot.scatter('z', 'y', marker='.')
plt.gca().set_aspect(1);
plt.tight_layout()

In [ ]:
plot_distribution(df, "py", "pz")

## View f

In [ ]:
import bgk

In [ ]:
# 5a: 4 nodes, release -> FAILS unless 
# 5b: 1 node, release
# 5c: 4 nodes, debug -> WORKS
# path = '/mnt/lustre/IAM851/jm1667/psc-runs/tests/test7/' # worked
path = '/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/exact/B00.10-n256/'
params_record = bgk.backend.ParamsRecord(path)

inputFile = params_record.path_to_input
B = params_record.B0
maxStep = params_record.nmax
reversed = params_record.reversed

print(f"path={path}")
print(f"input={inputFile}")
print(f"max step={maxStep}")
print(f"B={B}")
print(f"reversed={reversed}")

In [ ]:
step = 0
t = bgk.Loader(path, engine="pscadios2", species_names=['e', 'i'])._get_xr_dataset("pfd", step).time
print(f"t={t}")

df = read_step(step)
input = bgk.Input(inputFile)

df["r"] = (df.y ** 2 + df.z ** 2) ** .5
df["v_phi"] = (df.pz * df.y - df.py * df.z) / df.r
df["v_rho"] = (df.py * df.y + df.pz * df.z) / df.r
df.v_phi.fillna(0, inplace=True)
df.v_rho.fillna(0, inplace=True)
# df["w"] = (df.py ** 2 + df.pz ** 2) / 2 - df.r.map(lambda r: input.interpolate_value(r, "Psi"))
# df["l"] = 2 * df.r * df.v_phi - B * df.r ** 2

In [ ]:
# df = df[abs(df.v_phi) > 10]
df[abs(df.v_phi) > .1]

In [ ]:
hist, rhos, v_phis = np.histogram2d(df.r, df.v_phi, bins=[60,80])

v_phis_cc = (v_phis[1:] + v_phis[:-1]) / 2
rhos_cc = (rhos[1:] + rhos[:-1]) / 2
fs2d = hist.T / rhos_cc

mean_v_phis = fs2d.T.dot(v_phis_cc) / fs2d.sum(axis=0)
mean_v_phis_input = np.array([input.interpolate_value(rho, "v_phi") for rho in rhos_cc])

In [ ]:
# f(rho, v_phi)
plt.close("all")
%matplotlib widget
plt.pcolormesh(rhos, v_phis, fs2d, cmap="Reds")
plt.xlabel("$\\rho$")
plt.ylabel("$v_\\phi$")
plt.plot(rhos_cc, mean_v_phis, 'k', label="actual mean")
plt.plot(rhos_cc, mean_v_phis_input, 'b', label="target mean")
plt.title(f"f($\\rho$, $v_\\phi$) at t={t:.3f} for $B={B}$")
plt.colorbar()
plt.legend()
# plt.ylim(-.003, .003)
pass # don't print any output

In [ ]:
%matplotlib widget
plot_distribution(df, "y", "z")
# plot_distribution(df, "r", "l")
# plot_distribution(df, "r", "l")
# plot_distribution(df, "r", "w")